# Mostra colunas disponiveis do .csv

In [27]:
import pandas as pd

data = pd.read_csv("do2021opendata.csv", sep=";", low_memory=False)

",".join(data.columns.tolist())

'CONTADOR,CODINST,ORIGEM,TIPOBITO,DTOBITO,HORAOBITO,NUMSUS,NATURAL,CODMUNNATU,DTNASC,IDADE,SEXO,RACACOR,ESTCIV,ESC,ESC2010,SERIESCFAL,OCUP,CODMUNRES,LOCOCOR,CODESTAB,CODMUNOCOR,IDADEMAE,ESCMAE,ESCMAE2010,SERIESCMAE,OCUPMAE,QTDFILVIVO,QTDFILMORT,GRAVIDEZ,SEMAGESTAC,GESTACAO,PARTO,OBITOPARTO,PESO,TPMORTEOCO,OBITOGRAV,OBITOPUERP,ASSISTMED,EXAME,CIRURGIA,NECROPSIA,LINHAA,LINHAB,LINHAC,LINHAD,LINHAII,CAUSABAS,CB_PRE,COMUNSVOIM,DTATESTADO,CIRCOBITO,ACIDTRAB,FONTE,NUMEROLOTE,DTINVESTIG,DTCADASTRO,ATESTANTE,STCODIFICA,CODIFICADO,VERSAOSIST,VERSAOSCB,FONTEINV,DTRECEBIM,ATESTADO,DTRECORIGA,OPOR_DO,CAUSAMAT,ESCMAEAGR1,ESCFALAGR1,STDOEPIDEM,STDONOVA,DIFDATA,NUDIASOBCO,DTCADINV,TPOBITOCOR,DTCONINV,FONTES,TPRESGINFO,TPNIVELINV,DTCADINF,MORTEPARTO,DTCONCASO,ALTCAUSA,CAUSABAS_O,TPPOS,TP_ALTERA,CB_ALT'

# Todos os casos de morte por covid em Joinville durante 2021

In [28]:
COD_MUN_JLLE = 420910
COD_COVID_1 = "B342"

jlle_data = data[data["CODMUNOCOR"] == COD_MUN_JLLE]
jlle_only_covid = jlle_data[jlle_data["CAUSABAS_O"] == COD_COVID_1]

# Construção da planilha de dados covid 2021

In [42]:
import csv
import numpy as np

def format_date(raw_date: str) -> str:
    date_length = len(raw_date)

    if date_length == 7:
        return "0" + raw_date[0:1] + "/" + raw_date[1:3] + "/" + raw_date[3:7]
    else:
        return raw_date[0:2] + "/" + raw_date[2:4] + "/" + raw_date[4:8]

def format_schooling(code: int) -> str:
    if code == 0:
        return "Sem escolaridade"
    elif code == 1:
        return "Ensino Fundamental (Anos Iniciais)"
    elif code == 2:
        return "Ensino Fundamental (Anos Finais)"
    elif code == 3:
        return "Ensino Médio"
    elif code == 4:
        return "Superior incompleto"
    elif code == 5:
        return "Superior completo"
    elif code == 9:
        return "Não informado"

def format_age(code: str) -> str:
    first_digit = code[0]
    second_digits = code [1:3]

    # minutos
    if first_digit == "1":
        return "0"
    # horas
    elif first_digit == "2":
        return "0"
    # meses
    elif first_digit == "3":
        return "0"
    # anos
    elif first_digit == "4":
        return second_digits
    # mais de 100 anos
    elif first_digit == "5":
        return "1" + second_digits
    else:
        return "Não informado"

def format_breed_or_color(code: int) -> str:
    if code == 1:
        return "Branca"
    elif code == 2:
        return "Preta"
    elif code == 3:
        return "Amarela"
    elif code == 4:
        return "Parda"
    elif code == 5:
        return "Indígena"

def format_sex(code: int) -> str:
    if code == 1:
        return "Masculino"
    elif code == 2:
        return "Feminino"
    else:
        return "Não informado"

def format_civil_status(code) -> str:
    if np.isnan(code):
        return "Não informado"

    if code == 1:
        return "Solteiro"
    elif code == 2:
        return "Casado"
    elif code == 3:
        return "Viúvo"
    elif code == 4:
        return "Separado judicialmente/divorciado"
    elif code == 5:
        return "União estável"
    else:
        return "Não informado"

with open('./covid_data_joinville_2021.csv', 'w+') as f:
    fieldnames = ['DATAOBITO', 'MUNICIPIO', 'DESCRICAO', 'ESCOLARIDADE', 'IDADE_ANOS', 'RACA_COR', 'SEXO', 'SITUACAO_CONJUGAL']
    dict_writer = csv.DictWriter(f, fieldnames)
    dict_writer.writeheader()

    for index, row in jlle_only_covid.iterrows():
        dt_obito = format_date(str(row['DTOBITO']))
        schooling = format_schooling(row['ESC2010'])
        age = format_age(str(row['IDADE']))
        breed_or_color = format_breed_or_color(int(row['RACACOR']))
        sex = format_sex(int(row['SEXO']))
        civil_status = format_civil_status(row['ESTCIV'])
        
        dict_writer.writerow({
            "DATAOBITO": dt_obito,
            "MUNICIPIO": "JOINVILLE",
            "DESCRICAO": "Infecção por coronavírus de localização não especificada",
            "ESCOLARIDADE": schooling,
            "IDADE_ANOS": age,
            "RACA_COR": breed_or_color,
            "SEXO": sex,
            "SITUACAO_CONJUGAL": civil_status
        })